# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
print("hi")

hi


In [3]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -----------------------------
# 1️⃣ Load your dataset
# -----------------------------
df = pd.read_csv("/kaggle/input/loanyyy-pca/loan_final_normalized_done _d.csv")  # change file if needed

TARGET = "loan_status"   # <-- change if your target column is different

X = df.drop(TARGET, axis=1)
y = df[TARGET].astype(int)

print("Original dimension:", X.shape[1])

# -----------------------------
# 2️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# -----------------------------
# 3️⃣ XGBoost Model (500 Iterations)
# -----------------------------
model = XGBClassifier(
    n_estimators=500,          # <-- 500 boosting rounds
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# -----------------------------
# 4️⃣ Predictions
# -----------------------------
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

# -----------------------------
# 5️⃣ Results
# -----------------------------
print("\n===== XGBoost Results (500 Iterations) =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 19

===== XGBoost Results (500 Iterations) =====
Test Accuracy : 0.92720000
Precision     : 0.92613738
Recall        : 0.94296094
F1 Score      : 0.93447345


In [4]:
df.shape

(50000, 20)

pca loan

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from xgboost import XGBClassifier   # 🔥 NEW

df = pd.read_csv("/kaggle/input/loanyyy-pca/loan_final_normalized_done _d.csv")

X = df.drop("loan_status",axis=1)
y = df["loan_status"].astype(int)

# 1) Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2) PCA to retain 95% variance
pca = PCA(n_components=0.95)  
X_pca = pca.fit_transform(X_scaled)

print("Original dimension:", X.shape[1])
print("PCA dimension:", X_pca.shape[1])

# 3) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.20, stratify=y, random_state=42
)

# ---------------------------------------------------
# 4) ❗ Replace CatBoost with XGBoost
# ---------------------------------------------------
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist"        # Fast training
)

# Fit
model.fit(X_train, y_train)

# Test prediction
y_pred = model.predict(X_test)

# Metrics
test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n=== PCA + XGBOOST MODEL RESULTS ===")
print("Test Accuracy :", test_acc)
print("Precision     :", test_prec)
print("Recall        :", test_rec)
print("F1 Score      :", test_f1)


Original dimension: 19
PCA dimension: 14

=== PCA + XGBOOST MODEL RESULTS ===
Test Accuracy : 0.8905
Precision     : 0.8879310344827587
Recall        : 0.916802906448683
F1 Score      : 0.9021360264545535


chi square

In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # ← changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as your hybrid pipeline)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Scale to non-negative for chi-square
# -------------------------
scaler = MinMaxScaler()   # maps features to [0, 1]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------
# 3. Try different numbers of selected features
# -------------------------
k_values = [5, 10, 15,11,13,15]

results_chi2 = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"CHI-SQUARE + XGBoost with top-{k} features")
    print("="*60)

    # 3.1 Chi-Square feature selection
    selector = SelectKBest(score_func=chi2, k=k)
    X_train_k = selector.fit_transform(X_train_scaled, y_train)
    X_test_k = selector.transform(X_test_scaled)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 3.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label encoder warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 3.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_chi2[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Chi-Square + XGBoost ===========")
for k, info in results_chi2.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 19

CHI-SQUARE + XGBoost with top-5 features
Selected 5 features:
['age', 'credit_history_years', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks']
Test Accuracy : 0.728600
Precision     : 0.718765
Recall        : 0.832879
F1 Score      : 0.771626

CHI-SQUARE + XGBoost with top-10 features
Selected 10 features:
['age', 'years_employed', 'credit_score', 'credit_history_years', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'product_type', 'debt_to_income_ratio', 'payment_to_income_ratio']
Test Accuracy : 0.887700
Precision     : 0.889858
Recall        : 0.908447
F1 Score      : 0.899056

CHI-SQUARE + XGBoost with top-15 features
Selected 15 features:
['age', 'years_employed', 'annual_income', 'credit_score', 'credit_history_years', 'savings_assets', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'product_type', 'loan_intent', 'interest_rate', 'debt_to_income_ratio', 'loan_to_income_ratio', 'payment_to_income_ra

mutual information

In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # <-- changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (keep same style as other experiments)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Try different numbers of selected features
# -------------------------
k_values = [5,10,15,11,13]

results_mi = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"MUTUAL INFORMATION + XGBoost with top-{k} features")
    print("="*60)

    # 2.1 Mutual Information feature selection
    selector = SelectKBest(score_func=mutual_info_classif, k=k)
    X_train_k = selector.fit_transform(X_train, y_train)
    X_test_k = selector.transform(X_test)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 2.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label-encoder warning in older xgboost
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 2.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_mi[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Mutual Information + XGBoost ===========")
for k, info in results_mi.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 19

MUTUAL INFORMATION + XGBoost with top-5 features
Selected 5 features:
['age', 'credit_score', 'delinquencies_last_2yrs', 'interest_rate', 'debt_to_income_ratio']
Test Accuracy : 0.843900
Precision     : 0.841176
Recall        : 0.883197
F1 Score      : 0.861675

MUTUAL INFORMATION + XGBoost with top-10 features
Selected 10 features:
['age', 'years_employed', 'credit_score', 'credit_history_years', 'savings_assets', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'interest_rate', 'debt_to_income_ratio']
Test Accuracy : 0.864600
Precision     : 0.866891
Recall        : 0.890827
F1 Score      : 0.878696

MUTUAL INFORMATION + XGBoost with top-15 features
Selected 15 features:
['age', 'years_employed', 'annual_income', 'credit_score', 'credit_history_years', 'savings_assets', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'product_type', 'loan_intent', 'interest_rate', 'debt_to_income_ratio', 'loan_to_income_ratio', 'payment_to_

recursive feature elimination

In [9]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # <-- changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as other baselines)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Try different numbers of selected features with RFE
# -------------------------
k_values = [5,10,15,11,13]

results_rfe = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"RFE (XGBoost) with top-{k} features")
    print("="*60)

    # Base estimator for RFE (reduced iterations to save time)
    base_model = XGBClassifier(
        n_estimators=200,            # reduced for speed during RFE
        learning_rate=0.05,
        max_depth=6,
        objective="binary:logistic",
        use_label_encoder=False,     # suppress older xgboost warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    # 2.1 RFE setup
    selector = RFE(
        estimator=base_model,
        n_features_to_select=k,
        step=1
    )

    # Fit RFE on training data
    selector.fit(X_train, y_train)

    # Transform train and test sets
    X_train_k = selector.transform(X_train)
    X_test_k = selector.transform(X_test)

    # Get selected feature names
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 2.2 Train a fresh XGBoost on the selected features (for fair comparison)
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 2.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_rfe[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: RFE (XGBoost) ===========")
for k, info in results_rfe.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 19

RFE (XGBoost) with top-5 features
Selected 5 features:
['credit_score', 'defaults_on_file', 'delinquencies_last_2yrs', 'loan_intent', 'debt_to_income_ratio']
Test Accuracy : 0.854900
Precision     : 0.850692
Recall        : 0.893188
F1 Score      : 0.871422

RFE (XGBoost) with top-10 features
Selected 10 features:
['credit_score', 'credit_history_years', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'product_type', 'loan_intent', 'debt_to_income_ratio', 'loan_to_income_ratio', 'payment_to_income_ratio']
Test Accuracy : 0.912900
Precision     : 0.912131
Recall        : 0.931517
F1 Score      : 0.921722

RFE (XGBoost) with top-15 features
Selected 15 features:
['age', 'occupation_status', 'years_employed', 'credit_score', 'credit_history_years', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'product_type', 'loan_intent', 'loan_amount', 'interest_rate', 'debt_to_income_ratio', 'loan_to_income_ratio', 'payment_to_income_rati

l1 lasso feature selection + xgboost

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier   # <-- changed
import numpy as np
import pandas as pd


feature_names = X.columns.tolist()

print("Original dimension:", X.shape[1])

# Train Logistic regression with L1 penalty to select features
log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=0.3, max_iter=2000)
log_reg.fit(X, y)

# Get non-zero coefficients
coef = log_reg.coef_[0]
selected_idx = np.where(coef != 0)[0]
selected_features = [feature_names[i] for i in selected_idx]

print("\nSelected features using L1 (Lasso):")
print(selected_features)
print("Total selected:", len(selected_features))

# ----- XGBoost on selected features -----

X_sel = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y, test_size=0.20, stratify=y, random_state=42
)

model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    use_label_encoder=False,   # suppress older xgboost label-encoder warning
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n===== L1 (Lasso) + XGBoost Results =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 19

Selected features using L1 (Lasso):
['customer_id', 'age', 'occupation_status', 'years_employed', 'annual_income', 'credit_score', 'credit_history_years', 'savings_assets', 'current_debt', 'defaults_on_file', 'delinquencies_last_2yrs', 'derogatory_marks', 'product_type', 'loan_intent', 'loan_amount', 'interest_rate', 'debt_to_income_ratio', 'loan_to_income_ratio', 'payment_to_income_ratio']
Total selected: 19

===== L1 (Lasso) + XGBoost Results =====
Test Accuracy : 0.92720000
Precision     : 0.92613738
Recall        : 0.94296094
F1 Score      : 0.93447345


xgboost importance + xgboost baseline

In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier

# -------------------------
# 1. Load data
# -------------------------

feature_names = X.columns.tolist()
print("Original dimension:", X.shape[1])

# -------------------------
# 2. Train XGBoost on ALL features to get importance
# -------------------------
base_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

base_model.fit(X, y)

# Get feature importances from XGBoost
importances = base_model.feature_importances_
# Sort indices by importance (descending)
sorted_idx = np.argsort(importances)[::-1]
sorted_features = [feature_names[i] for i in sorted_idx]

print("\nTop 20 features by XGBoost importance:")
for i in range(min(20, len(sorted_features))):
    print(f"{i+1:2d}. {sorted_features[i]}  (importance={importances[sorted_idx[i]]:.6f})")

# -------------------------
# 3. Evaluate XGBoost using top-k important features
# -------------------------
K_values = [5, 10, 15, 11,13]
results_xgb_imp = {}

print("\n============================================================")
print("XGBOOST FEATURE IMPORTANCE + XGBOOST BASELINE")
print("============================================================")

for k in K_values:
    top_k_features = sorted_features[:k]
    X_sel = X[top_k_features]

    # Same train-test strategy as other baselines
    X_train, X_test, y_train, y_test = train_test_split(
        X_sel, y,
        test_size=0.20,
        stratify=y,
        random_state=42
    )

    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"\n============================================================")
    print(f"XGBoost-Importance + XGBoost with top-{k} features")
    print("============================================================")
    print(f"Selected {k} features:")
    print(top_k_features)
    print(f"Test Accuracy : {acc:.8f}")
    print(f"Precision     : {prec:.8f}")
    print(f"Recall        : {rec:.8f}")
    print(f"F1 Score      : {f1:.8f}")

    results_xgb_imp[k] = {
        "features": top_k_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: XGBoost Importance + XGBoost ===========")
for k, info in results_xgb_imp.items():
    print(f"Top-{k} features -> F1={info['f1']:.8f}, Acc={info['acc']:.8f}")


Original dimension: 19

Top 20 features by XGBoost importance:
 1. defaults_on_file  (importance=0.199084)
 2. credit_score  (importance=0.165578)
 3. debt_to_income_ratio  (importance=0.103600)
 4. delinquencies_last_2yrs  (importance=0.099482)
 5. product_type  (importance=0.074467)
 6. loan_intent  (importance=0.067603)
 7. payment_to_income_ratio  (importance=0.060539)
 8. credit_history_years  (importance=0.046555)
 9. derogatory_marks  (importance=0.042836)
10. loan_to_income_ratio  (importance=0.037167)
11. years_employed  (importance=0.026586)
12. loan_amount  (importance=0.016354)
13. occupation_status  (importance=0.011668)
14. interest_rate  (importance=0.011019)
15. age  (importance=0.009721)
16. savings_assets  (importance=0.008047)
17. annual_income  (importance=0.007739)
18. current_debt  (importance=0.006255)
19. customer_id  (importance=0.005702)

XGBOOST FEATURE IMPORTANCE + XGBOOST BASELINE

XGBoost-Importance + XGBoost with top-5 features
Selected 5 features:
['defa

heart attack


pca

In [3]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -----------------------------
# 1️⃣ Load your dataset
# -----------------------------

df = pd.read_csv("/kaggle/input/heart-attack-smoted/heart_attack_smoted.csv")

TARGET = "Heart_Attack"   # <-- change if your target column is different

X = df.drop(TARGET, axis=1)
y = df[TARGET].astype(int)

print("Original dimension:", X.shape[1])

# -----------------------------
# 2️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# -----------------------------
# 3️⃣ XGBoost Model (500 Iterations)
# -----------------------------
model = XGBClassifier(
    n_estimators=500,          # <-- 500 boosting rounds
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# -----------------------------
# 4️⃣ Predictions
# -----------------------------
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

# -----------------------------
# 5️⃣ Results
# -----------------------------
print("\n===== XGBoost Results (500 Iterations) =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 28

===== XGBoost Results (500 Iterations) =====
Test Accuracy : 0.92628060
Precision     : 1.00000000
Recall        : 0.85256120
F1 Score      : 0.92041353


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from xgboost import XGBClassifier   # 🔥 NEW

df = pd.read_csv("/kaggle/input/heart-attack-smoted/heart_attack_smoted.csv")

X = df.drop("Heart_Attack",axis=1)
y = df["Heart_Attack"].astype(int)

# 1) Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2) PCA to retain 95% variance
pca = PCA(n_components=0.95)  
X_pca = pca.fit_transform(X_scaled)

print("Original dimension:", X.shape[1])
print("PCA dimension:", X_pca.shape[1])

# 3) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.20, stratify=y, random_state=42
)

# ---------------------------------------------------
# 4) ❗ Replace CatBoost with XGBoost
# ---------------------------------------------------
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist"        # Fast training
)

# Fit
model.fit(X_train, y_train)

# Test prediction
y_pred = model.predict(X_test)

# Metrics
test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n=== PCA + XGBOOST MODEL RESULTS ===")
print("Test Accuracy :", test_acc)
print("Precision     :", test_prec)
print("Recall        :", test_rec)
print("F1 Score      :", test_f1)


Original dimension: 28
PCA dimension: 27

=== PCA + XGBOOST MODEL RESULTS ===
Test Accuracy : 0.7612194016319129
Precision     : 0.7462606837606838
Recall        : 0.7915911151405258
F1 Score      : 0.7682578090629125


chi square

In [15]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # ← changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as your hybrid pipeline)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Scale to non-negative for chi-square
# -------------------------
scaler = MinMaxScaler()   # maps features to [0, 1]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------
# 3. Try different numbers of selected features
# -------------------------
k_values = [5, 10, 15, 20, 4,8]

results_chi2 = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"CHI-SQUARE + XGBoost with top-{k} features")
    print("="*60)

    # 3.1 Chi-Square feature selection
    selector = SelectKBest(score_func=chi2, k=k)
    X_train_k = selector.fit_transform(X_train_scaled, y_train)
    X_test_k = selector.transform(X_test_scaled)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 3.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label encoder warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 3.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_chi2[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Chi-Square + XGBoost ===========")
for k, info in results_chi2.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 28

CHI-SQUARE + XGBoost with top-5 features
Selected 5 features:
['Diabetes', 'Angina', 'Heart_Disease_History', 'Medication', 'Obesity']
Test Accuracy : 0.620014
Precision     : 0.611380
Recall        : 0.658772
F1 Score      : 0.634192

CHI-SQUARE + XGBoost with top-10 features
Selected 10 features:
['Gender', 'Smoking', 'Diabetes', 'Family_History', 'Angina', 'Heart_Disease_History', 'Diet', 'Urban_Rural', 'Medication', 'Obesity']
Test Accuracy : 0.636446
Precision     : 0.619944
Recall        : 0.705236
F1 Score      : 0.659845

CHI-SQUARE + XGBoost with top-15 features
Selected 15 features:
['Gender', 'Region', 'Smoking', 'Diabetes', 'Family_History', 'Angina', 'Heart_Disease_History', 'Diet', 'Occupation', 'Income_Level', 'Education_Level', 'Marital_Status', 'Urban_Rural', 'Medication', 'Obesity']
Test Accuracy : 0.692486
Precision     : 0.668352
Recall        : 0.764166
F1 Score      : 0.713055

CHI-SQUARE + XGBoost with top-20 features
Selected 20 features:

mutual information

In [16]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # <-- changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (keep same style as other experiments)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Try different numbers of selected features
# -------------------------
k_values = [5,10,15,20,4,8]

results_mi = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"MUTUAL INFORMATION + XGBoost with top-{k} features")
    print("="*60)

    # 2.1 Mutual Information feature selection
    selector = SelectKBest(score_func=mutual_info_classif, k=k)
    X_train_k = selector.fit_transform(X_train, y_train)
    X_test_k = selector.transform(X_test)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 2.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label-encoder warning in older xgboost
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 2.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_mi[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Mutual Information + XGBoost ===========")
for k, info in results_mi.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 28

MUTUAL INFORMATION + XGBoost with top-5 features
Selected 5 features:
['Age', 'BMI', 'Heart_Rate', 'Sleep_Hours', 'Daily_Water_Intake']
Test Accuracy : 0.921691
Precision     : 1.000000
Recall        : 0.843382
F1 Score      : 0.915038

MUTUAL INFORMATION + XGBoost with top-10 features
Selected 10 features:
['Age', 'Blood_Pressure', 'Cholesterol', 'BMI', 'Heart_Rate', 'Stress_Level', 'Sleep_Hours', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health']
Test Accuracy : 0.926167
Precision     : 0.999867
Recall        : 0.852448
F1 Score      : 0.920291

MUTUAL INFORMATION + XGBoost with top-15 features
Selected 15 features:
['Age', 'Blood_Pressure', 'Cholesterol', 'BMI', 'Heart_Rate', 'Diabetes', 'Stress_Level', 'Heart_Disease_History', 'Sleep_Hours', 'Physical_Activity', 'Marital_Status', 'Medication', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health']
Test Accuracy : 0.926337
Precision     : 1.000000
Recall        : 0.852675
F1 Score      : 0.920480


recursive feature elimination

In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # <-- changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as other baselines)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Try different numbers of selected features with RFE
# -------------------------
k_values = [5,10,15,20,4,8]

results_rfe = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"RFE (XGBoost) with top-{k} features")
    print("="*60)

    # Base estimator for RFE (reduced iterations to save time)
    base_model = XGBClassifier(
        n_estimators=200,            # reduced for speed during RFE
        learning_rate=0.05,
        max_depth=6,
        objective="binary:logistic",
        use_label_encoder=False,     # suppress older xgboost warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    # 2.1 RFE setup
    selector = RFE(
        estimator=base_model,
        n_features_to_select=k,
        step=1
    )

    # Fit RFE on training data
    selector.fit(X_train, y_train)

    # Transform train and test sets
    X_train_k = selector.transform(X_train)
    X_test_k = selector.transform(X_test)

    # Get selected feature names
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 2.2 Train a fresh XGBoost on the selected features (for fair comparison)
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 2.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_rfe[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: RFE (XGBoost) ===========")
for k, info in results_rfe.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 28

RFE (XGBoost) with top-5 features
Selected 5 features:
['Diabetes', 'Stress_Level', 'Heart_Disease_History', 'Health_Awareness', 'Mental_Health']
Test Accuracy : 0.925714
Precision     : 1.000000
Recall        : 0.851428
F1 Score      : 0.919753

RFE (XGBoost) with top-10 features
Selected 10 features:
['Gender', 'Diabetes', 'Stress_Level', 'Angina', 'Heart_Disease_History', 'Medication', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health', 'Obesity']
Test Accuracy : 0.925941
Precision     : 0.999867
Recall        : 0.851995
F1 Score      : 0.920027

RFE (XGBoost) with top-15 features
Selected 15 features:
['Age', 'Gender', 'Heart_Rate', 'Smoking', 'Diabetes', 'Family_History', 'Stress_Level', 'Angina', 'Heart_Disease_History', 'Sleep_Hours', 'Medication', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health', 'Obesity']
Test Accuracy : 0.926734
Precision     : 1.000000
Recall        : 0.853468
F1 Score      : 0.920942

RFE (XGBoost) with top-20 featur

l1 lasso feature selection + xgboost

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier   # <-- changed
import numpy as np
import pandas as pd


feature_names = X.columns.tolist()

print("Original dimension:", X.shape[1])

# Train Logistic regression with L1 penalty to select features
log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=0.3, max_iter=2000)
log_reg.fit(X, y)

# Get non-zero coefficients
coef = log_reg.coef_[0]
selected_idx = np.where(coef != 0)[0]
selected_features = [feature_names[i] for i in selected_idx]

print("\nSelected features using L1 (Lasso):")
print(selected_features)
print("Total selected:", len(selected_features))

# ----- XGBoost on selected features -----

X_sel = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y, test_size=0.20, stratify=y, random_state=42
)

model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    use_label_encoder=False,   # suppress older xgboost label-encoder warning
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n===== L1 (Lasso) + XGBoost Results =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 28

Selected features using L1 (Lasso):
['Age', 'Gender', 'Region', 'Blood_Pressure', 'Cholesterol', 'BMI', 'Heart_Rate', 'Exercise_Level', 'Smoking', 'Alcohol_Consumption', 'Diabetes', 'Family_History', 'Stress_Level', 'Angina', 'Heart_Disease_History', 'Diet', 'Sleep_Hours', 'Occupation', 'Income_Level', 'Physical_Activity', 'Education_Level', 'Marital_Status', 'Urban_Rural', 'Medication', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health', 'Obesity']
Total selected: 28

===== L1 (Lasso) + XGBoost Results =====
Test Accuracy : 0.92628060
Precision     : 1.00000000
Recall        : 0.85256120
F1 Score      : 0.92041353


xgboost importance + xgboost

In [20]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier

# -------------------------
# 1. Load data
# -------------------------

feature_names = X.columns.tolist()
print("Original dimension:", X.shape[1])

# -------------------------
# 2. Train XGBoost on ALL features to get importance
# -------------------------
base_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

base_model.fit(X, y)

# Get feature importances from XGBoost
importances = base_model.feature_importances_
# Sort indices by importance (descending)
sorted_idx = np.argsort(importances)[::-1]
sorted_features = [feature_names[i] for i in sorted_idx]

print("\nTop 20 features by XGBoost importance:")
for i in range(min(20, len(sorted_features))):
    print(f"{i+1:2d}. {sorted_features[i]}  (importance={importances[sorted_idx[i]]:.6f})")

# -------------------------
# 3. Evaluate XGBoost using top-k important features
# -------------------------
K_values = [5, 10, 15, 20,4,8]
results_xgb_imp = {}

print("\n============================================================")
print("XGBOOST FEATURE IMPORTANCE + XGBOOST BASELINE")
print("============================================================")

for k in K_values:
    top_k_features = sorted_features[:k]
    X_sel = X[top_k_features]

    # Same train-test strategy as other baselines
    X_train, X_test, y_train, y_test = train_test_split(
        X_sel, y,
        test_size=0.20,
        stratify=y,
        random_state=42
    )

    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"\n============================================================")
    print(f"XGBoost-Importance + XGBoost with top-{k} features")
    print("============================================================")
    print(f"Selected {k} features:")
    print(top_k_features)
    print(f"Test Accuracy : {acc:.8f}")
    print(f"Precision     : {prec:.8f}")
    print(f"Recall        : {rec:.8f}")
    print(f"F1 Score      : {f1:.8f}")

    results_xgb_imp[k] = {
        "features": top_k_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: XGBoost Importance + XGBoost ===========")
for k, info in results_xgb_imp.items():
    print(f"Top-{k} features -> F1={info['f1']:.8f}, Acc={info['acc']:.8f}")


Original dimension: 28

Top 20 features by XGBoost importance:
 1. Health_Awareness  (importance=0.390064)
 2. Stress_Level  (importance=0.152729)
 3. Mental_Health  (importance=0.152697)
 4. Diabetes  (importance=0.038637)
 5. Heart_Disease_History  (importance=0.031578)
 6. Medication  (importance=0.030690)
 7. Angina  (importance=0.021057)
 8. Obesity  (importance=0.019911)
 9. Daily_Water_Intake  (importance=0.013725)
10. Gender  (importance=0.013648)
11. Heart_Rate  (importance=0.013400)
12. Family_History  (importance=0.012155)
13. Age  (importance=0.010369)
14. Sleep_Hours  (importance=0.009569)
15. Marital_Status  (importance=0.007509)
16. Smoking  (importance=0.007483)
17. Diet  (importance=0.006898)
18. Income_Level  (importance=0.006882)
19. Physical_Activity  (importance=0.006649)
20. Education_Level  (importance=0.006305)

XGBOOST FEATURE IMPORTANCE + XGBOOST BASELINE

XGBoost-Importance + XGBoost with top-5 features
Selected 5 features:
['Health_Awareness', 'Stress_Level'

2019-ciciddos_full



In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -----------------------------
# 1️⃣ Load your dataset
# -----------------------------
df = pd.read_csv("/kaggle/input/cicid-2019-cleaned/cicddos2019_cleaned_final.csv")  # change file if needed

TARGET = "Label"   # <-- change if your target column is different

X = df.drop(TARGET, axis=1)
y = df[TARGET].astype(int)

print("Original dimension:", X.shape[1])

# -----------------------------
# 2️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# -----------------------------
# 3️⃣ XGBoost Model (500 Iterations)
# -----------------------------
model = XGBClassifier(
    n_estimators=500,          # <-- 500 boosting rounds
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# -----------------------------
# 4️⃣ Predictions
# -----------------------------
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

# -----------------------------
# 5️⃣ Results
# -----------------------------
print("\n===== XGBoost Results (500 Iterations) =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 77

===== XGBoost Results (500 Iterations) =====
Test Accuracy : 0.99885000
Precision     : 0.99939934
Recall        : 0.99830000
F1 Score      : 0.99884937


pca

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from xgboost import XGBClassifier   # 🔥 NEW

df = pd.read_csv("/kaggle/input/cicid-2019-cleaned/cicddos2019_cleaned_final.csv")

X = df.drop("Label",axis=1)
y = df["Label"].astype(int)

# 1) Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2) PCA to retain 95% variance
pca = PCA(n_components=0.95)  
X_pca = pca.fit_transform(X_scaled)

print("Original dimension:", X.shape[1])
print("PCA dimension:", X_pca.shape[1])

# 3) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.20, stratify=y, random_state=42
)

# ---------------------------------------------------
# 4) ❗ Replace CatBoost with XGBoost
# ---------------------------------------------------
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist"        # Fast training
)

# Fit
model.fit(X_train, y_train)

# Test prediction
y_pred = model.predict(X_test)

# Metrics
test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n=== PCA + XGBOOST MODEL RESULTS ===")
print("Test Accuracy :", test_acc)
print("Precision     :", test_prec)
print("Recall        :", test_rec)
print("F1 Score      :", test_f1)


Original dimension: 77
PCA dimension: 24

=== PCA + XGBOOST MODEL RESULTS ===
Test Accuracy : 0.9985
Precision     : 0.9987992795677406
Recall        : 0.9982
F1 Score      : 0.9984995498649595


chi square

In [26]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # ← changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as your hybrid pipeline)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Scale to non-negative for chi-square
# -------------------------
scaler = MinMaxScaler()   # maps features to [0, 1]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------
# 3. Try different numbers of selected features
# -------------------------
k_values = [5, 10, 15, 20, 30,31,9]

results_chi2 = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"CHI-SQUARE + XGBoost with top-{k} features")
    print("="*60)

    # 3.1 Chi-Square feature selection
    selector = SelectKBest(score_func=chi2, k=k)
    X_train_k = selector.fit_transform(X_train_scaled, y_train)
    X_test_k = selector.transform(X_test_scaled)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 3.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label encoder warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 3.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_chi2[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Chi-Square + XGBoost ===========")
for k, info in results_chi2.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 77

CHI-SQUARE + XGBoost with top-5 features
Selected 5 features:
['Fwd Packet Length Min', 'Packet Length Min', 'URG Flag Count', 'CWE Flag Count', 'Avg Fwd Segment Size']
Test Accuracy : 0.967400
Precision     : 0.945907
Recall        : 0.991500
F1 Score      : 0.968167

CHI-SQUARE + XGBoost with top-10 features
Selected 10 features:
['Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Flow Bytes/s', 'Fwd PSH Flags', 'Packet Length Min', 'RST Flag Count', 'URG Flag Count', 'CWE Flag Count', 'Avg Packet Size', 'Avg Fwd Segment Size']
Test Accuracy : 0.970700
Precision     : 0.947178
Recall        : 0.997000
F1 Score      : 0.971451

CHI-SQUARE + XGBoost with top-15 features
Selected 15 features:
['Protocol', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Flow Bytes/s', 'Flow Packets/s', 'Fwd PSH Flags', 'Fwd Packets/s', 'Packet Length Min', 'Packet Length Mean', 'RST Flag Count', 'URG Flag Count', 'CWE Flag Count', 'Avg Packet Size', 'Avg Fwd Segment Size', 'I

mutual information

In [27]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # <-- changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (keep same style as other experiments)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Try different numbers of selected features
# -------------------------
k_values = [5, 10, 15, 20, 30,31,9]

results_mi = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"MUTUAL INFORMATION + XGBoost with top-{k} features")
    print("="*60)

    # 2.1 Mutual Information feature selection
    selector = SelectKBest(score_func=mutual_info_classif, k=k)
    X_train_k = selector.fit_transform(X_train, y_train)
    X_test_k = selector.transform(X_test)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 2.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label-encoder warning in older xgboost
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 2.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_mi[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Mutual Information + XGBoost ===========")
for k, info in results_mi.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 77

MUTUAL INFORMATION + XGBoost with top-5 features
Selected 5 features:
['Fwd Packet Length Mean', 'Packet Length Min', 'Packet Length Mean', 'Avg Packet Size', 'Avg Fwd Segment Size']
Test Accuracy : 0.958750
Precision     : 0.987669
Recall        : 0.929100
F1 Score      : 0.957490

MUTUAL INFORMATION + XGBoost with top-10 features
Selected 10 features:
['Fwd Packets Length Total', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Flow Bytes/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Avg Packet Size', 'Avg Fwd Segment Size', 'Subflow Fwd Bytes']
Test Accuracy : 0.964250
Precision     : 0.998604
Recall        : 0.929800
F1 Score      : 0.962974

MUTUAL INFORMATION + XGBoost with top-15 features
Selected 15 features:
['Fwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Max', 'Fwd Packets/s', 'Packet Length Min', 'Packet Lengt

recursive feature elimination

In [28]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # <-- changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as other baselines)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Try different numbers of selected features with RFE
# -------------------------
k_values = [5, 10, 15, 20, 30,31,9]

results_rfe = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"RFE (XGBoost) with top-{k} features")
    print("="*60)

    # Base estimator for RFE (reduced iterations to save time)
    base_model = XGBClassifier(
        n_estimators=200,            # reduced for speed during RFE
        learning_rate=0.05,
        max_depth=6,
        objective="binary:logistic",
        use_label_encoder=False,     # suppress older xgboost warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    # 2.1 RFE setup
    selector = RFE(
        estimator=base_model,
        n_features_to_select=k,
        step=1
    )

    # Fit RFE on training data
    selector.fit(X_train, y_train)

    # Transform train and test sets
    X_train_k = selector.transform(X_train)
    X_test_k = selector.transform(X_test)

    # Get selected feature names
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 2.2 Train a fresh XGBoost on the selected features (for fair comparison)
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 2.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_rfe[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: RFE (XGBoost) ===========")
for k, info in results_rfe.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 77

RFE (XGBoost) with top-5 features
Selected 5 features:
['Flow IAT Mean', 'Packet Length Min', 'ACK Flag Count', 'URG Flag Count', 'Init Fwd Win Bytes']
Test Accuracy : 0.998050
Precision     : 0.998499
Recall        : 0.997600
F1 Score      : 0.998049

RFE (XGBoost) with top-10 features
Selected 10 features:
['Protocol', 'Fwd Packet Length Min', 'Flow IAT Mean', 'Flow IAT Min', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Std', 'ACK Flag Count', 'URG Flag Count', 'Init Fwd Win Bytes']
Test Accuracy : 0.998650
Precision     : 0.998999
Recall        : 0.998300
F1 Score      : 0.998650

RFE (XGBoost) with top-15 features
Selected 15 features:
['Protocol', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Std', 'ACK Flag Count', 'URG Flag Count', 'Init Fwd Win Bytes', 'Init Bwd Win Bytes']
Test Accuracy : 0.998950
Precision

l1 lasso feature selection + xgboost

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier   # <-- changed
import numpy as np
import pandas as pd


feature_names = X.columns.tolist()

print("Original dimension:", X.shape[1])

# Train Logistic regression with L1 penalty to select features
log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=0.3, max_iter=2000)
log_reg.fit(X, y)

# Get non-zero coefficients
coef = log_reg.coef_[0]
selected_idx = np.where(coef != 0)[0]
selected_features = [feature_names[i] for i in selected_idx]

print("\nSelected features using L1 (Lasso):")
print(selected_features)
print("Total selected:", len(selected_features))

# ----- XGBoost on selected features -----

X_sel = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y, test_size=0.20, stratify=y, random_state=42
)

model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    use_label_encoder=False,   # suppress older xgboost label-encoder warning
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n===== L1 (Lasso) + XGBoost Results =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 77

Selected features using L1 (Lasso):
['Protocol', 'Flow Duration', 'Fwd Packet Length Std', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Packets/s', 'Flow IAT Std', 'Fwd IAT Mean', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Max', 'Fwd Header Length', 'Bwd Header Length', 'Packet Length Min', 'RST Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'Down/Up Ratio', 'Avg Packet Size', 'Avg Bwd Segment Size', 'Init Fwd Win Bytes', 'Init Bwd Win Bytes', 'Fwd Seg Size Min', 'Active Mean', 'Active Min', 'Idle Min']
Total selected: 29

===== L1 (Lasso) + XGBoost Results =====
Test Accuracy : 0.99895000
Precision     : 0.99939946
Recall        : 0.99850000
F1 Score      : 0.99894953


xgboost importance+baseline xgboost

In [29]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier

# -------------------------
# 1. Load data
# -------------------------

feature_names = X.columns.tolist()
print("Original dimension:", X.shape[1])

# -------------------------
# 2. Train XGBoost on ALL features to get importance
# -------------------------
base_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

base_model.fit(X, y)

# Get feature importances from XGBoost
importances = base_model.feature_importances_
# Sort indices by importance (descending)
sorted_idx = np.argsort(importances)[::-1]
sorted_features = [feature_names[i] for i in sorted_idx]

print("\nTop 20 features by XGBoost importance:")
for i in range(min(20, len(sorted_features))):
    print(f"{i+1:2d}. {sorted_features[i]}  (importance={importances[sorted_idx[i]]:.6f})")

# -------------------------
# 3. Evaluate XGBoost using top-k important features
# -------------------------
K_values = [5, 10, 15, 20, 30,31,9]
results_xgb_imp = {}

print("\n============================================================")
print("XGBOOST FEATURE IMPORTANCE + XGBOOST BASELINE")
print("============================================================")

for k in K_values:
    top_k_features = sorted_features[:k]
    X_sel = X[top_k_features]

    # Same train-test strategy as other baselines
    X_train, X_test, y_train, y_test = train_test_split(
        X_sel, y,
        test_size=0.20,
        stratify=y,
        random_state=42
    )

    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"\n============================================================")
    print(f"XGBoost-Importance + XGBoost with top-{k} features")
    print("============================================================")
    print(f"Selected {k} features:")
    print(top_k_features)
    print(f"Test Accuracy : {acc:.8f}")
    print(f"Precision     : {prec:.8f}")
    print(f"Recall        : {rec:.8f}")
    print(f"F1 Score      : {f1:.8f}")

    results_xgb_imp[k] = {
        "features": top_k_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: XGBoost Importance + XGBoost ===========")
for k, info in results_xgb_imp.items():
    print(f"Top-{k} features -> F1={info['f1']:.8f}, Acc={info['acc']:.8f}")


Original dimension: 77

Top 20 features by XGBoost importance:
 1. Packet Length Min  (importance=0.656341)
 2. Flow IAT Mean  (importance=0.162800)
 3. ACK Flag Count  (importance=0.047233)
 4. URG Flag Count  (importance=0.037925)
 5. Init Fwd Win Bytes  (importance=0.015062)
 6. Flow Packets/s  (importance=0.011742)
 7. Packet Length Std  (importance=0.010502)
 8. Bwd Packets/s  (importance=0.009584)
 9. Fwd Packet Length Min  (importance=0.008364)
10. Flow IAT Min  (importance=0.003687)
11. Bwd Packet Length Max  (importance=0.003624)
12. Bwd Packets Length Total  (importance=0.002982)
13. Flow Bytes/s  (importance=0.002641)
14. Idle Std  (importance=0.001901)
15. Protocol  (importance=0.001832)
16. Idle Max  (importance=0.001560)
17. Fwd Packets/s  (importance=0.001433)
18. Flow IAT Std  (importance=0.001269)
19. Flow IAT Max  (importance=0.001268)
20. Fwd Packet Length Max  (importance=0.001200)

XGBOOST FEATURE IMPORTANCE + XGBOOST BASELINE

XGBoost-Importance + XGBoost with top